In [ ]:
from ffast import load
from windML import RNN

tokeniser = load('poincare')
SENTENCE_VECTOR_SIZE = 1240
POINCARE_TOKEN_VECTOR_SIZE = 100

token_ids_vectoriser = lambda token_ids: list(map(
    lambda token_vector:token_vector.reshape((-1,1)),
    tokeniser.decode(token_ids).semantics()
))
encoder = lambda text: tokeniser.encode(text).vector.reshape(-1,1)

decoder = RNN(token_vector_size=POINCARE_TOKEN_VECTOR_SIZE,token_vocabulary_size=len(tokeniser)+4,hidden_dimension=SENTENCE_VECTOR_SIZE)

examples = ["i love you","i hate you"]
token_ids = list(map(lambda example:tokeniser.encode(example).ids,examples))
contexts = ["positive","negative"]
context_vectors = list(map(encoder,contexts))

decoder.fit(token_ids_vectoriser, token_ids)
decoder.save("bla")
generated_token_ids = decoder.generate(token_ids_vectoriser,prompt_ids=[],condition_vector=encoder("positive"))
sentence = str(tokeniser.decode(generated_token_ids))
print(sentence)